In [4]:
from dotenv import load_dotenv
import os
import websockets
import json
load_dotenv()
HUME_API_KEY = os.getenv("HUME_API_KEY")

transcript = """
Hi, I... I’m bleeding really badly. I need help.
"""

In [5]:
websocket_url = f"wss://api.hume.ai/v0/stream/models"

In [17]:
async with websockets.connect(websocket_url, extra_headers={"X-Hume-Api-Key": HUME_API_KEY}) as websocket: 
    await websocket.send(json.dumps({
        "data": transcript,
        "models": {
            "language": {}
        },
        "raw_text": True
        }))
    data = await websocket.recv()
    response_data = json.loads(data)
    # Aggregating emotions across all words and calculating the average score for each emotion
    emotions_data = response_data.get("language", {}).get("predictions", [])
    emotion_scores = {}
    total_words = len(emotions_data)

    for word_data in emotions_data:
        word_emotions = word_data.get("emotions", [])
        for emotion in word_emotions:
            if emotion['name'] in emotion_scores:
                emotion_scores[emotion['name']] += emotion['score']
            else:
                emotion_scores[emotion['name']] = emotion['score']

    # Calculating average scores
    for emotion in emotion_scores:
        emotion_scores[emotion] /= total_words

    # Sorting and selecting the top three emotions for the entire text chunk
    top_three_overall_emotions = sorted(emotion_scores.items(), key=lambda item: item[1], reverse=True)[:3]
    print(transcript)
    print("Top Three Emotions for the Entire Text:", top_three_overall_emotions)



Hi, I... I’m bleeding really badly. I need help.

Top Three Emotions for the Entire Text: [('Pain', 0.5748002013650775), ('Distress', 0.4501630634121183), ('Sadness', 0.20735129240589836)]
